In [156]:
import os
import requests 
from bs4 import BeautifulSoup 
import re 
import pandas as pd

Create a CSV file in which scraped data is saved

In [157]:
directory = '/Users/abha/Desktop/GitHub/Lending Club'
#directory = './GitHub/Lending Club'
try:
    os.makedirs(directory)
except OSError:
    if not os.path.isdir(directory):
        raise
os.chdir(directory)

Here, I am only focusing on Notes that have been issued and sold prior to 12-31-2010. However, the analysis can be easily extented for entire sample period uptil 8-31-2017.

In [158]:
def get_filing_links(url):
    'Function that identifies and collects all hyperlinks displayed on URL into a list'
    r = requests.get(url)
    links = re.findall('<filingHREF>(.*?)</filingHREF>', r.text)
    links_new = [link.replace('-index.htm', '.txt') for link in links]
    return (links_new)    

In [159]:
def get_sales_filing(url):
    'Function that checks if URL is to a sales filing or not'
    response = requests.get(url) #FILENAME is either salessup or postingsup
    if re.search('salessup', response.text) is not None:
        print('sales prospectus found')
        success = 1
    else:
        success=0
    return (success)   

In [160]:
def get_scraped_data(url):
    '''Function that scrapes info on borrower FICO scores, loan request date and final issuance date 
    for each note sold in Sales filings'''
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    data = []
    data1 = []
    data2 = []
    for tag in soup.find_all(text=re.compile('Series of Member Payment Dependent Notes')):
        if tag.findParent('table') is not None:
            table = tag.findParent('table')
            content = table.findNext('tr').findNext('tr').get_text().split('\n\n')[1:]
            content = [s.rstrip() for s in content]
            header = ['Series of Member Payment Dependent Notes', 'Aggregate principal amount of Notes offered', 
                      'Aggregate principal amount of Notes sold', 'Stated interest rate', 'Service charge', 
                      'Sale and Original Issue Date', 'Initial Maturity', 'Final Maturity', 
                      'Amount of corresponding member loan funded by Lending Club']
            data.append(dict(zip(header, content))) #create list of dictionaries
    for tag in soup.find_all(text=re.compile('Credit Score Range:')):
        if tag.findParent('table') is not None:
            table = tag.findParent('table')
            content = table.tr.findNext('td').findNext('td').get_text()
            header = ['Credit score range']
            data1.append(dict(zip(header, [content])))
    for tag in soup.find_all(text=re.compile(r'was requested on.*by a borrower')): 
            content = [re.findall('was requested on (.*?) by', tag),
                       url.replace('.txt', '-index.htm')]
            header = ['Request date', 'File name']
            data2.append(dict(zip(header, content)))       
    return (data, data1, data2)

In [161]:
cik = "0001409970"  
sec_filing = "424B3"
priorto = "20101231"
count = 100
for file_num in range(1, 16):
    print('Opening CSV file %d for writing!' %file_num)
    file_name = 'Sales Report %d.csv' %(file_num)
    output = open(file_name, 'w')
    start = (file_num - 1)*100
    payload = {'CIK': cik, 'type': sec_filing, 'dateb':priorto, 'owner':'exclude', 'start': str(0), 
          'count':str(count), 'output':'xml'}
    base_url = requests.get('https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany', params=payload).url
    filing_links = get_filing_links(base_url)
    ctr = 0
    for url in filing_links:
        success = get_sales_filing(url)
        if success ==1:
            ctr += 1
            (data, data1, data2) = get_scraped_data(url)
            if ctr==1:
                df = pd.concat([pd.concat([pd.DataFrame(data), pd.DataFrame(data1)], axis=1), pd.DataFrame(data2)], axis=1)
            else:
                a = pd.concat([pd.concat([pd.DataFrame(data), pd.DataFrame(data1)], axis=1), pd.DataFrame(data2)], axis=1)
                df = pd.concat([df, a])         
    df.to_csv(file_name, index=False)
    output.close()  

Opening CSV file 1 for writing!
sales prospectus found
sales prospectus found
sales prospectus found
sales prospectus found
Opening CSV file 2 for writing!
sales prospectus found
sales prospectus found
sales prospectus found
sales prospectus found
Opening CSV file 3 for writing!
sales prospectus found
sales prospectus found
sales prospectus found
sales prospectus found
Opening CSV file 4 for writing!
sales prospectus found
sales prospectus found
sales prospectus found
sales prospectus found
Opening CSV file 5 for writing!
sales prospectus found
sales prospectus found
sales prospectus found
sales prospectus found
Opening CSV file 6 for writing!
sales prospectus found
sales prospectus found
sales prospectus found
sales prospectus found
Opening CSV file 7 for writing!
sales prospectus found
sales prospectus found
sales prospectus found
sales prospectus found
Opening CSV file 8 for writing!
sales prospectus found
sales prospectus found
sales prospectus found
sales prospectus found
Opening 